In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error as MSE
import warnings
warnings.filterwarnings("ignore")
import lightautoml

In [2]:
###
# ДЗ ML3:
# 1. Датасет titanic, разделить на train/test в пропорции 0.7/0.3
# 2. Предсказать возраст Age на основании других параметров (отберите сами какие)
# 3. Подобрать оптимальные гиперпараметры модели с помощью GridSearch, в идеале получить лучшее качество на test
###

In [3]:
## Резюме

# Результат линейной регрессии на тесте после оптимизации (MSE):  175.53050966534144
# Лучшие параметры регрессии: {'fit_intercept': True, 'positive': False}

# Лучший показатель DecisionTreeRegressor на тесте при оптимизации (MSE):  205.57545932291464
# Лучшие параметры дерева: {'max_depth': 2, 'min_samples_leaf': 9, 'min_samples_split': 2}

# Лучший показатель XGBRegressor на тесте при оптимизации (MSE):  201.8597682322448
# Лучшие параметры:  {'colsample_bytree': 0.0, 'gamma': 0.0, 'learning_rate': 0.35, 'max_depth': 4, 'min_child_weight': 7}

# Лучший показатель LAML на тесте при оптимизации (MSE):  196.89774507441547
# Cпецификация
# Final prediction for new objects (level 0) = 
#	 0.66269 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
#	 0.33731 * (5 averaged models Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost) 


In [4]:
raw_data = pd.read_csv('https://raw.githubusercontent.com/RMA1011/da_hw/main/ML/HW%203/titanic.csv')
RANDOM_STATE = 42

In [5]:
#### Обработка данных
raw_data.dropna(inplace = True)
features = raw_data[['Survived','Pclass','Sex','SibSp','Fare']]
features['Sex'].replace({'male':0,'female':1}, inplace = True)
target = raw_data['Age']
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.3, random_state = RANDOM_STATE)

In [6]:
#### Пробуем линейную регрессию
lin_mod = LinearRegression()
fit_mod = LinearRegression().fit(X = X_train,y = y_train)
pred_lin = fit_mod.predict(X_test)
print('Результат линейной регрессии на тесте до оптимизации (MSE): ', MSE(y_test, pred_lin))
parameters = {'fit_intercept': [True,False],
'positive':[True, False]}
lin_grid = GridSearchCV(estimator = lin_mod, scoring = 'neg_mean_squared_error',param_grid = parameters, cv = 5, n_jobs =-1, refit = True)
lin_grid = lin_grid.fit(X_train, y_train)
best_lin_model = lin_grid.best_estimator_
lin_pred = best_lin_model.predict(X_test)

print('Результат линейной регрессии на обучении после оптимизации (MSE): ', -1*lin_grid.best_score_)
print('Результат линейной регрессии на тесте после оптимизации (MSE): ', MSE(y_test, lin_pred))
print('Лучшие параметры: ', lin_grid.best_params_)

Результат линейной регрессии на тесте до оптимизации (MSE):  175.53050966534144
Результат линейной регрессии на обучении после оптимизации (MSE):  239.40141875015624
Результат линейной регрессии на тесте после оптимизации (MSE):  175.53050966534144
Лучшие параметры:  {'fit_intercept': True, 'positive': False}


In [7]:
#### Пробуем дерево
decis_tree = DecisionTreeRegressor(random_state=42)
parameters_tree = {
    'max_depth':  [1,2,3,4,5,6,7,8,9,10],
    'min_samples_split': [2,3,4,5,6,7,8,9,10],
    'min_samples_leaf': [2,3,4,5,6,7,8,9,10]
}

fit_tree = decis_tree.fit(X_train, y_train)
fit_tree_pred = fit_tree.predict(X_test)
print('Лучший показатель на обучении до оптимизации (MSE): ', MSE(y_test, fit_tree_pred))

decis_grid = GridSearchCV(
    estimator = decis_tree, 
    param_grid = parameters_tree,
    scoring = 'neg_mean_squared_error',
    cv = 5,
    refit = True     
)

decis_grid = decis_grid.fit(X_train, y_train)
print('Лучшие параметры: ',decis_grid.best_params_)
print('Лучший показатель на обучении при оптимизации (MSE): ', -1*decis_grid.best_score_)
best_decis_model = decis_grid.best_estimator_
best_decis_model = best_decis_model.fit(X_train,y_train)
best_predicted = best_decis_model.predict(X_test)
print('Лучший показатель на тесте при оптимизации (MSE): ', MSE(y_test, best_predicted))

Лучший показатель на обучении до оптимизации (MSE):  342.5092072727272
Лучшие параметры:  {'max_depth': 2, 'min_samples_leaf': 9, 'min_samples_split': 2}
Лучший показатель на обучении при оптимизации (MSE):  227.80952042290446
Лучший показатель на тесте при оптимизации (MSE):  205.57545932291464


In [14]:
### Пробуем xgboost!

xgb_r = xgb.XGBRegressor(objective ='reg:squarederror',
                  n_estimators = 10, seed = RANDOM_STATE)

xgb_r.fit(X_train, y_train)
xgb_pred = xgb_r.predict(X_test)
print("Результат на тесте до оптимизации (MSE) : % f" %(MSE(y_test, xgb_pred)))

parameters_xgb = [
    {
    'learning_rate': (0.01,0.10,0.2,0.35,0.5),
    'max_depth': [3,4,5,6,7],
    'min_child_weight': [1,2,3,4,5,6,7],
    'gamma': [0.0,0.1,0.2,0.3,0.4,0.5],
    'colsample_bytree': [0.0,0.1,0.2,0.3,0.4,0.5]
    }
]

xgb_grid = GridSearchCV(
    estimator = xgb_r, 
    param_grid = parameters_xgb,
    scoring = 'neg_mean_squared_error',
    cv = 5,
    refit = True)

xgb_grid = xgb_grid.fit(X_train, y_train)
print('Лучшие параметры: ',xgb_grid.best_params_)
print('Лучший показатель на обучении при оптимизации (MSE): ', -1*xgb_grid.best_score_)
best_xgb_model = xgb_grid.best_estimator_
best_xgb_model = best_xgb_model.fit(X_train,y_train)
xgb_opt_pred = best_xgb_model.predict(X_test)
print('Лучший показатель на тесте при оптимизации (MSE): ', MSE(y_test, xgb_opt_pred))

Результат на тесте до оптимизации (MSE) :  208.103283
Лучшие параметры:  {'colsample_bytree': 0.0, 'gamma': 0.0, 'learning_rate': 0.35, 'max_depth': 4, 'min_child_weight': 7}
Лучший показатель на обучении при оптимизации (MSE):  229.63715166508436
Лучший показатель на тесте при оптимизации (MSE):  201.8597682322448


In [28]:
### Пробуем LightAutoML
### пример: https://colab.research.google.com/github/AILab-MLTools/LightAutoML/blob/master/examples/tutorials/Tutorial_1_basics.ipynb#scrollTo=yl0IidgAD1_e
### https://colab.research.google.com/github/AILab-MLTools/LightAutoML/blob/master/examples/tutorials/Tutorial_2_WhiteBox_AutoWoE.ipynb

from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
from lightautoml.report.report_deco import ReportDeco

roles = {'target': 'Age'}
task = Task(name = 'reg', metric = 'mse')
data_automl = raw_data[['Age','Survived','Pclass','Sex','SibSp','Fare']]
data_automl['Sex'].replace({'male':0,'female':1}, inplace = True)

automl_model = TabularAutoML(
task = task, reader_params= {'random_state': RANDOM_STATE, 'cv':5}
)

tr_data_automl, te_data_automl = train_test_split(
    data_automl,
    test_size=0.3, # из технического задания
    #stratify= data_automl['Age'],
    random_state=RANDOM_STATE
)

oof_pred_automl = automl_model.fit_predict(tr_data_automl, roles = roles, verbose = 1)
te_pred_automl = automl_model.predict(te_data_automl)
print(f'Результат LAML на обучении: ',MSE(tr_data_automl['Age'].values, oof_pred_automl.data[:, 0]))
print(f'Результат LAML на тесте: ',MSE(te_data_automl['Age'].values, te_pred_automl.data[:, 0]))
print(automl_model.create_model_str_desc())



[08:02:12] Stdout logging level is INFO.
[08:02:12] Task: reg

[08:02:12] Start automl preset with listed constraints:
[08:02:12] - time: 3600.00 seconds
[08:02:12] - CPU: 4 cores
[08:02:12] - memory: 16 GB

[08:02:12] Train data shape: (128, 6)

[08:02:19] Layer 1 train process start. Time left 3593.45 secs
[08:02:19] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[08:02:21] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = -205.84605895582203
[08:02:21] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[08:02:21] Time left 3590.78 secs

[08:02:22] Selector_LightGBM fitting and predicting completed
[08:02:22] Start fitting Lvl_0_Pipe_1_Mod_0_LightGBM ...
[08:02:22] Fitting Lvl_0_Pipe_1_Mod_0_LightGBM finished. score = -217.48509883350934
[08:02:22] Lvl_0_Pipe_1_Mod_0_LightGBM fitting and predicting completed
[08:02:22] Start hyperparameters optimization for Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM ... Time budget is 300.00 secs
[08:02:37] Hyperparameters optimization for Lvl_0_

In [29]:
#print(automl_model.create_model_str_desc())


Final prediction for new objects (level 0) = 
	 0.66269 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.33731 * (5 averaged models Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost) 
